# Import Libraries

In [1]:
import os
import io
from pdf2image import convert_from_path
from pdfrw import PdfWriter, PdfReader
import ocr
# import utility
import json
from docx import Document
import shutil

# Get current path and append sys path

In [2]:
current_path = os.getcwd()

path_env_var = os.environ.get("PATH")
updated_path_env_var = f"{current_path}\\poppler\\Library\\bin;{path_env_var}"
print(updated_path_env_var)
os.environ['PATH'] = updated_path_env_var

d:\JJPES\new\scrap\poppler\Library\bin;c:\Users\admin\AppData\Local\Programs\Python\Python311;c:\Users\admin\AppData\Roaming\Python\Python311\Scripts;C:\WINDOWS\system32;C:\WINDOWS;C:\WINDOWS\System32\Wbem;C:\WINDOWS\System32\WindowsPowerShell\v1.0\;C:\Users\admin\AppData\Roaming\nvm;C:\Program Files\nodejs;C:\Program Files\Git\cmd;C:\WINDOWS\System32\OpenSSH\;;C:\Program Files\Docker\Docker\resources\bin;C:\Users\admin\AppData\Local\Programs\Python\Python311\Scripts\;C:\Users\admin\AppData\Local\Programs\Python\Python311\;C:\Users\admin\AppData\Local\Microsoft\WindowsApps;C:\Users\admin\AppData\Local\Programs\Microsoft VS Code\bin;C:\Users\admin\AppData\Roaming\nvm;C:\Program Files\nodejs;C:\Users\admin\AppData\Local\Programs\EmEditor;D:\TOOLS\vs_BuildTools.exe;C:\Program Files\JetBrains\PyCharm 2023.2.1\bin;;C:\WINDOWS\system32;C:\WINDOWS;C:\WINDOWS\System32\Wbem;C:\WINDOWS\System32\WindowsPowerShell\v1.0\;C:\Users\admin\AppData\Roaming\nvm;C:\Program Files\nodejs;C:\Program Files\Gi

# Get pdf files in the current directory

In [3]:
files = [current_path + "\\" + f for f in os.listdir('./') if f.endswith('.pdf')]
print(files)

['d:\\JJPES\\new\\scrap\\merged.pdf']


# Set Deed, Mortgage, Judgment, Liens

In [4]:
doc_index = 0

deed_array = ['Warranty Deed', 'Quit Claim Deed', 'QuitClaim Deed', 'Disclaimer Deed', 'Special Warranty Deed', 'Grant Deed', 'Trustees Deed upon Sale', "Trustee's Deed", 'Limited Warranty Deed', 'Bargain and sale deed', 'Administrator deeds', 'Sheriff Deed', 'Deed Under Power Of Sale', 'General Warranty Deed', 'North Carolina General Warranty Deed', 'This Deed', 'Deed', 'Release Deed']
deed_array = [deed.lower() for deed in deed_array]
mortgage_array = ['Deed of Trust', 'Security Deed', 'Mortgage', 'Partial Claims Mortgage', 'Short form Deed of Trust']
mortgage_array = [mortgage.lower() for mortgage in mortgage_array]
judgment_array = ['Abstract Judgment', 'Divorce Decree', 'Criminal Restitution Judgment', 'Notice of Lis Pendens', 'FIFA', 'Final Judgment']
judgment_array = [judgment.lower() for judgment in judgment_array]
liens_array = ['Water Lien', 'Sewer Lien', 'Federal Tax Lien', 'State Tax Lien', 'UCC Financing Statement', 'Homeowners Association Lien', 'Mechanic Lien', 'Notice and Claim of Lien', 'Tax Lien', 'Code Violation Lien', 'Notice of Pending Lien', 'Claim Of Lien', 'Notice Regarding Payment Of Support', 'Notice Of Federal Tax Lien', 'Statement Of Lien']
liens_array = [liens.lower() for liens in liens_array]
assignments_array = ['Assignment of Security Deed', 'Corporate Assignment of Deed of Trust', 'Assignment of Security Instrument', 'Assignment of First Mortgage and Assignment of Rents', 'Assignment of Mortgage']
assignments_array = [assignments.lower() for assignments in assignments_array]
modification_array = ['Loan Modification Agreement', 'Mortgage Modification Agreement', 'Agreement of Modification of Mortgage', 'Modification Agreement', 'FHA Home Affordable Modification Agreement']
modification_array = [modification.lower() for modification in modification_array]
substitution_array = ['Substitution Of Trustee', 'Appointment Of Substitute Trustees', 'Deed Of Appointment Of Substitute Trustee']
substitution_array = [substitution.lower() for substitution in substitution_array]

# List out all the directories

In [5]:
dirs = os.listdir('./')
print(dirs)

['bot.ipynb', 'bot.py', 'Documents Parsing', 'merged.pdf', 'ocr.py', 'poppler', '__pycache__']


# Handle pdf files

In [8]:
for file in files:
    filename = file.split('\\')[-1].split('.')[0]
    print(f'-------------- {filename} ----------------')
    reader = PdfReader(file)
    
    images = convert_from_path(file)
    # print(images)
    final_str = ""
    parcel_flag = 0
    count_judg = 1
    count_assign = 1
    count_financial = 1
    deeds_str = ""
    additional_str = ""
    num_legal = 0
    prev_page_deed = 0
    deed_txt = ""
    financial_txt = ""
    prev_page_financial = 0
    
    doc_names = []
    pages = []
    for i, image in enumerate(images):
        img_bytes = io.BytesIO()
        image.save(img_bytes, format='PNG')
        
        content = img_bytes.getvalue()
        
        # Run the OCR on the image
        result = ocr.textract_analysis(content)
        # print(result)
        
        str_str = ""
        flag = 0
        reason = ""
        deed_flag = 0
        assign_flag = 0
        legal_flag = 0
        
        for sstr in result:
            str_str += sstr
            str_str += " "
        lower_str = str_str.lower()
        
        core_content = ""
        if len(lower_str) > 200:
            core_content = lower_str[:200]
        else:
            core_content = lower_str
        print('--------------------------------------------------------')
        if "Uniform Residential Loan Application".lower() in core_content:
            doc_names.append("Uniform Residential Loan Application")
            pages.append(i)
        if "Uniform Residential Appraisal Report".lower() in core_content:
            doc_names.append("Uniform Residential Appraisal Report")
            pages.append(i)
        if "Individual Income Tax Return".lower() in core_content:
            doc_names.append("Individual Income Tax Return")
            pages.append(i)
        if "Bank of America".lower() in core_content:
            doc_names.append("Bank of America")
            pages.append(i)
        if "Closing Disclosure".lower() in core_content:
            doc_names.append("Closing Disclosure")
            pages.append(i)
        if "Credit Report".lower() in core_content:
            doc_names.append("Credit Report")
            pages.append(i)
        if "Settlement Statement".lower() in core_content:
            doc_names.append("Settlement Statement")
            pages.append(i)
        if "PAY STUB".lower() in core_content:
            doc_names.append("PAY STUB")
            pages.append(i)
        if "Approval Letter".lower() in core_content:
            doc_names.append("Approval Letter")
            pages.append(i)
        if "22222".lower() in core_content:
            doc_names.append("22222")
            pages.append(i)

    print(doc_names)
    print(pages)
    for i in range(len(pages)):
        reader = PdfReader(file)
        pdf_writer = PdfWriter()
        
        # If it's the last index item, include all remaining pages
        if i == len(pages) - 1:
            for j in range(pages[i], len(reader.pages)):
                pages = reader.pages[j]
                pdf_writer.addpage(new_func())
        else:
            for j in range(pages[i], pages[i + 1]):
                page = reader.pages[j]
                pdf_writer.addpage(page)
                
        # Write the split PDF file
        with open(doc_names[i] + '_' + str(i+1) + '.pdf', 'wb') as output_pdf:
            pdf_writer.write(output_pdf)
            
    
    

-------------- merged ----------------
--------------------------------------------------------
--------------------------------------------------------
--------------------------------------------------------
--------------------------------------------------------
--------------------------------------------------------
--------------------------------------------------------
--------------------------------------------------------
--------------------------------------------------------
--------------------------------------------------------
--------------------------------------------------------
--------------------------------------------------------
--------------------------------------------------------
--------------------------------------------------------
--------------------------------------------------------
--------------------------------------------------------
['Uniform Residential Loan Application', 'Uniform Residential Loan Application', 'Uniform Residential Loan

[WARNING] tokens.py:221 Did not find PDF object (639, 0) (line=99897, col=1, token='endobj')
[WARNING] tokens.py:221 Did not find PDF object (640, 0) (line=99897, col=1, token='endobj')
[WARNING] tokens.py:221 Did not find PDF object (654, 0) (line=99897, col=1, token='endobj')
[WARNING] tokens.py:221 Did not find PDF object (655, 0) (line=99897, col=1, token='endobj')
[WARNING] tokens.py:221 Did not find PDF object (657, 0) (line=107128, col=1, token='endobj')
[WARNING] tokens.py:221 Did not find PDF object (658, 0) (line=107128, col=1, token='endobj')
[WARNING] tokens.py:221 Did not find PDF object (660, 0) (line=107267, col=1, token='endobj')
[WARNING] tokens.py:221 Did not find PDF object (661, 0) (line=107267, col=1, token='endobj')
[WARNING] tokens.py:221 Did not find PDF object (648, 0) (line=107267, col=1, token='endobj')
[WARNING] tokens.py:221 Did not find PDF object (649, 0) (line=107267, col=1, token='endobj')
[WARNING] tokens.py:221 Did not find PDF object (663, 0) (line=1